import random
import pygame

In [50]:
import random
import pygame
import time



class button():
    def __init__(self,color,x,y,width,height,tsize,text=''):
        self.color=color
        self.x=x
        self.y=y
        self.width=width
        self.height=height
        self.text=text
        self.tsize=tsize

    def drawbutton(self,WIN,outline=None):
        if outline:
            pygame.draw.rect(WIN,outline,(self.x-2,self.y-2,self.width+4,self.height+4),0)
        pygame.draw.rect(WIN,self.color,(self.x,self.y,self.width,self.height),0)
        if self.text != '':
            myfont = pygame.font.SysFont('Comic Sans MS', self.tsize)
            text = myfont.render(self.text, 1, (0, 0, 0))
            WIN.blit(text,(self.x+(self.width/2-text.get_width()/2),self.y+(self.height/2-text.get_height()/2)))

   

    def isOverSquare(self,pos):
        if pos[0]>self.x and pos[0]<self.x+self.width:
            if pos[1]>self.y and pos[1]<self.y+self.height:
                return True
        return False


class minesweeper():
    
    def __init__(self,r,c,m,level=None,color=None):
        self.row=r
        self.col=c
        self.mines=m
        self.level=level
        self.color=color
        self.play=True
        self.numflags=0
        self.seenBoard=[]
        self.nonSeenBoard=[]
        self.gamesplayed=0
        self.gameswon=0
        self.buttons=[]
        self.createSeenBoard()
        self.createNonSeenBoard()
        self.addMines()
        self.numberEverything()
      
    def printSeenBoard(self):
        print("printing seenBoard")
        for i in range(self.row):
            line=""
            for j in range(self.col):
                line+=str(self.seenBoard[i][j])
                line+=" "
            print(line)
        print()

    def printNonSeenBoard(self):
        print("printing nonSeenBoard")
        for i in range(self.row):
            line=""
            for j in range(self.col):
                line+=str(self.nonSeenBoard[i][j])
                line+=" "
            print(line)
        print()
    
    def createSeenBoard(self):
        for i in range(self.row):
            r=[]
            for j in range(self.col):
                r.append(' ')
            self.seenBoard.append(r)
    
    def createNonSeenBoard(self):
        for i in range(self.row):
            r=[]
            for j in range(self.col):
                r.append(' ')
            self.nonSeenBoard.append(r)

    def addMines(self):
        count =0
        maxx=self.row*self.col-1
        while count<self.mines:
            num=random.randint(0, maxx)
            r=num//self.col
            c=num%self.col
            if self.nonSeenBoard[r][c]=='*':
                continue
            else:
                self.nonSeenBoard[r][c]='*'
                count+=1
    
    def numberEverything(self):
        for i in range(self.row):
            for j in range(self.col):
                if self.nonSeenBoard[i][j]!='*':
                    self.countNeighbors(i,j)

    def countNeighbors(self,r,c):
        count=0
        for i in range(r-1,r+2):
            for j in range(c-1,c+2):
                if i>=0 and i<self.row and j>=0 and j<self.col:
                    if i==r and j==c:
                        continue
                    else:
                        if self.nonSeenBoard[i][j]=='*':
                            count+=1
        self.nonSeenBoard[r][c]=count

    def dig(self,r,c):
        if r>=0 and c>=0 and r<self.row and c<self.col:
            if self.seenBoard[r][c]==' ':
                if self.nonSeenBoard[r][c]=='*':
                    print("game over")
                    txt=["you lost :(","better luck next time"]
                    endOfGame(WIN,self,txt,0)
                elif self.nonSeenBoard[r][c]==0:
                    self.seenBoard[r][c]=self.nonSeenBoard[r][c]
                    self.digNeighbors(r,c)
                    
                    num=self.countSeenBoard()
                    if num+self.mines==self.row*self.col:
                        print("you won")
                        txt=["congrats!","you won!!"]
                        endOfGame(WIN,self,txt,1)
                        
                else:
                    self.seenBoard[r][c]=self.nonSeenBoard[r][c]
                    num=self.countSeenBoard()
                    if num+self.mines==self.row*self.col:
                        print("you won")
                        txt=["congrats!","you won!!"]
                        endOfGame(WIN,self,txt,1)
                        
    def digNeighbors(self,r,c):
        for i in range(r-1, r+2):
            for j in range(c-1,c+2):
                if i>=0 and i<self.row and j>=0 and j<self.col:
                    if i==r and j==c:
                        continue
                    else:
                        if self.seenBoard[i][j]==' ':
                            if self.nonSeenBoard[i][j]=='*':
                                continue
                            elif self.nonSeenBoard[i][j]==0:
                                self.seenBoard[i][j]=self.nonSeenBoard[i][j]
                                self.digNeighbors(i,j)
                            else:
                                self.seenBoard[i][j]=self.nonSeenBoard[i][j]                                

    def countSeenBoard(self)->int:
        count=0
        for i in range(self.row):
            for j in range(self.col):
                if self.seenBoard[i][j]!=' ' and self.seenBoard[i][j]!='f':
                    count+=1
        return count
    
    def markFlag(self,r,c):
        if self.numflags<self.mines:
            if r>=0 and c>=0 and r<self.row and c<self.col:
                if self.seenBoard[r][c]==' ':
                    self.seenBoard[r][c]='f' 
                    self.numflags+=1
        if self.numflags==self.mines:
            self.checkFlags()
    
    def unmarkFlag(self,r,c):
        if self.numflags>0:
            if r>=0 and c>=0 and r<self.row and c<self.col:
                if self.seenBoard[r][c]=='f':
                    self.seenBoard[r][c]=' ' 
                    self.numflags-=1
    
    def checkFlags(self):
        count=0
        for i in range(self.row):
            for j in range(self.col):
                if self.nonSeenBoard[i][j]=='*' and self.seenBoard[i][j]=='f':
                    count+=1
        if count== self.mines:
            print("you won")
            txt=["congrats!","you won!!"]
            endOfGame(WIN,self,txt,1)
            
    def restart(self):
        self.nonSeenBoard=[]
        self.seenBoard=[]
        self.createSeenBoard()
        self.createNonSeenBoard()
        self.addMines()
        self.numberEverything()
        self.play=True
        self.numflags=0
        self.buttons=[]



def endOfGame(WIN,game,txt,num):
    WIN.fill((50,150,255))
    w=int(WIDTH/game.col)
    h=int((HEIGHT-200)/game.row)
    game.gamesplayed+=1
    if num==1:
        game.gameswon+=1
        for i in range(game.row):
            for j in range(game.col):
                x=w*j
                y=h*i+100
                pygame.draw.rect(WIN,LIGHTGRAY,(x,y,w,h))
                pygame.draw.rect(WIN,BLACK,(x,y,w,h),2)
                if game.seenBoard[i][j]!=' ':
                    if game.seenBoard[i][j]!='f':
                        myfont = pygame.font.SysFont('Comic Sans MS', 25)
                        text = myfont.render(str(game.seenBoard[i][j]),1,COLORS[game.seenBoard[i][j]] )
                        WIN.blit(text,(x+(w/2-text.get_width()/2),y+(h/2-text.get_height()/2)))
                if game.seenBoard[i][j]=='f':
                    myfont = pygame.font.SysFont('Comic Sans MS', 25)
                    text = myfont.render(str(game.seenBoard[i][j]),1,BLACK )
                    WIN.blit(text,(x+(w/2-text.get_width()/2),y+(h/2-text.get_height()/2)))

    else:
        for i in range(game.row):
            for j in range(game.col):
                x=w*j
                y=h*i+100
                pygame.draw.rect(WIN,LIGHTGRAY,(x,y,w,h))
                pygame.draw.rect(WIN,BLACK,(x,y,w,h),2)
                if game.seenBoard[i][j]!=' ':
                    if game.seenBoard[i][j]!='f':
                        myfont = pygame.font.SysFont('Comic Sans MS', 25)
                        text = myfont.render(str(game.seenBoard[i][j]),1,COLORS[game.seenBoard[i][j]] )
                        WIN.blit(text,(x+(w/2-text.get_width()/2),y+(h/2-text.get_height()/2)))
                if game.nonSeenBoard[i][j]=='*':
                    myfont = pygame.font.SysFont('Comic Sans MS', 30)
                    text = myfont.render(str(game.nonSeenBoard[i][j]),1,RED )
                    WIN.blit(text,(x+(w/2-text.get_width()/2),y+(h/2-text.get_height()/2)))
                    myfont = pygame.font.SysFont('Comic Sans MS', 20)
                    text = myfont.render(str(game.nonSeenBoard[i][j]),1,BLACK )
                    WIN.blit(text,(x+(w/2-text.get_width()/2),y+(h/2-text.get_height()/2)))    
    if num==1:
        pygame.draw.rect(WIN,LIGHTGRAY,(150,10,200,85))
        for i in range(len(txt)):
            x=150
            y=20+i*30
            myfont = pygame.font.SysFont('Comic Sans MS', 30)
            text = myfont.render(txt[i],1,PURPLE)
            WIN.blit(text,(x+(200/2-text.get_width()/2),y+(30/2-text.get_height()/2)))
    else:
        pygame.draw.rect(WIN,LIGHTGRAY,(50,10,400,85)) 
        for i in range(len(txt)):
            x=50
            y=20+i*30
            myfont = pygame.font.SysFont('Comic Sans MS', 30)
            text = myfont.render(txt[i],1,PURPLE)
            WIN.blit(text,(x+(400/2-text.get_width()/2),y+(30/2-text.get_height()/2)))

    x=50
    y=620
    w=200
    h=50
    # pygame.draw.rect(WIN,LIGHTBLUE,(x,y,w,h))
    myfont = pygame.font.SysFont('Comic Sans MS', 20)
    txt="Games Played:   "
    txt+=str(game.gamesplayed)
    text = myfont.render(txt,1,BLACK )
    WIN.blit(text,(x+(w/2-text.get_width()/2),y+(h/2-text.get_height()/2)))

    x=250
    y=620
    w=200
    h=50
    # pygame.draw.rect(WIN,LIGHTBLUE,(x,y,w,h))
    myfont = pygame.font.SysFont('Comic Sans MS', 20)
    txt="Games Won:   "
    txt+=str(game.gameswon)
    text = myfont.render(txt,1,BLACK )
    WIN.blit(text,(x+(w/2-text.get_width()/2),y+(h/2-text.get_height()/2)))
    pygame.display.update()
    time.sleep(3)
    
    game.buttons.append(button(WHITE,50,300,400,300,30,"Do you want to play again?"))
    game.buttons.append(button(GREEN, 150,500,100,50,30,"yes"))
    game.buttons.append(button(RED, 300,500,100,50,30,"no"))

def redrawWindow(WIN,game):
    WIN.fill(LIGHTBLUE)
    w=int(WIDTH/game.col)
    h=int((HEIGHT-200)/game.row)
    
    for i in range(game.row):
        for j in range(game.col):
            x=w*j
            y=h*i+100
            pygame.draw.rect(WIN,LIGHTGRAY,(x,y,w,h))
            pygame.draw.rect(WIN,BLACK,(x,y,w,h),2)
            if game.seenBoard[i][j]!=' ':
                if game.seenBoard[i][j]!='f':
                    myfont = pygame.font.SysFont('Comic Sans MS', 25)
                    text = myfont.render(str(game.seenBoard[i][j]),1,COLORS[game.seenBoard[i][j]] )
                    WIN.blit(text,(x+(w/2-text.get_width()/2),y+(h/2-text.get_height()/2)))
            if game.seenBoard[i][j]=='f':
                myfont = pygame.font.SysFont('Comic Sans MS', 20)
                text = myfont.render(str(game.seenBoard[i][j]),1,BLACK )
                WIN.blit(text,(x+(w/2-text.get_width()/2),y+(h/2-text.get_height()/2)))
            if game.seenBoard[i][j]=='*' :
                myfont = pygame.font.SysFont('Comic Sans MS', 20)
                text = myfont.render(str(game.SeenBoard[i][j]),1,BLACK )
                WIN.blit(text,(x+(w/2-text.get_width()/2),y+(h/2-text.get_height()/2)))
    


    x=200
    y=30
    w=100
    h=50
    myfont = pygame.font.SysFont('Comic Sans MS', 40)
    text = myfont.render(game.level,1,game.color )
    WIN.blit(text,(x+(w/2-text.get_width()/2),y+(h/2-text.get_height()/2)))

    x=50
    y=610
    w=200
    h=40
    myfont = pygame.font.SysFont('Comic Sans MS', 20)
    txt="Games Played:   "
    txt+=str(game.gamesplayed)
    text = myfont.render(txt,1,BLACK )
    WIN.blit(text,(x+(w/2-text.get_width()/2),y+(h/2-text.get_height()/2)))

    x=250
    y=610
    w=200
    h=40
    myfont = pygame.font.SysFont('Comic Sans MS', 20)
    txt="Games Won:   "
    txt+=str(game.gameswon)
    text = myfont.render(txt,1,BLACK )
    WIN.blit(text,(x+(w/2-text.get_width()/2),y+(h/2-text.get_height()/2)))

    x=50
    y=650
    w=400
    h=40
    myfont = pygame.font.SysFont('Comic Sans MS', 20)
    txt="number of flags used:    "
    txt+=str(game.numflags)
    txt+=" / "
    txt+=str(game.mines)
    text = myfont.render(txt,1,BLACK )

    WIN.blit(text,(x+(w/2-text.get_width()/2),y+(h/2-text.get_height()/2)))
    
    if len(game.buttons)>1:
        for i in range(len(game.buttons)):
            game.buttons[i].drawbutton(WIN)
    pygame.display.update()
 
def getGridPosition(pos,game):
        c=pos[0]//int(WIDTH//game.col)
        r=(pos[1]-100)//int((HEIGHT-200)//game.row)
        return r,c


def pickLevel(WIN):
    WIN.fill(LIGHTBLUE)
    levels=[]
    exp=["10 rows, 10 cols, 10 mines","15 rows, 15 cols, 30 mines","20 rows, 20 cols, 50 mines"]
    easy=button(GREEN,125,150,250,100,40,"EASY")
    medium=button(YELLOW,125,300,250,100,40,"MEDIUM")
    hard=button(RED,125,450,250,100,40,"HARD")
    levels.append(easy)
    levels.append(medium)
    levels.append(hard)

    for i in range(len(levels)):
        pygame.draw.rect(WIN,levels[i].color,(levels[i].x,levels[i].y,levels[i].width,levels[i].height),0)
        myfont = pygame.font.SysFont('Comic Sans MS', 40)
        text=myfont.render(levels[i].text,1,BLACK )
        WIN.blit(text,(levels[i].x +(levels[i].width/2-text.get_width()/2),levels[i].y+(levels[i].height/3-text.get_height()/3)))
        myfont = pygame.font.SysFont('Comic Sans MS', 15)
        text = myfont.render(exp[i],1,BLACK )
        WIN.blit(text,(levels[i].x +(levels[i].width/2-text.get_width()/2),levels[i].y+((levels[i].height/3)*2-(text.get_height()/3))))
    
    pygame.display.update()
    return levels
    
    

pygame.init()
pygame.font.init()
WIDTH,HEIGHT =500,700  
WIN=pygame.display.set_mode((WIDTH,HEIGHT))
pygame.display.set_caption("MINESWEEPER")
PINK=(255,0,127)
RED=(255,0,0)
ORANGE=(255,185,0)
YELLOW=(255,255,0)
GREEN=(0,205,0)
CYAN=(0,255,255)
BLUE=(0,0,255)
PURPLE=(120,0,160)
WHITE=(255,255,255)
BLACK=(0,0,0)
LIGHTGRAY=(200,200,200)
# LIGHTBLUE=(50,150,255)
LIGHTBLUE=(150,200,255)
COLORS=[BLACK,RED,GREEN,BLUE,PURPLE,ORANGE,CYAN,YELLOW,PINK]

def main():
    start=True
    run=True 
    r=None
    c=None
    m=None
    level=None
    color=None
    while (start):
        l=pickLevel(WIN)
        for event in pygame.event.get():
            if event.type==pygame.QUIT:
                start=False
                run=False
                pygame.quit()           
            if event.type==pygame.MOUSEBUTTONDOWN:
                pos=pygame.mouse.get_pos()
                if l[0].isOverSquare(pos):
                    r=10
                    c=10
                    m=10
                    level="EASY"
                    color=GREEN
                    start=False
                elif l[1].isOverSquare(pos):
                    r=15
                    c=15
                    m=30
                    level="MEDIUM"
                    color=YELLOW
                    start=False
                elif l[2].isOverSquare(pos):
                    r=20
                    c=20
                    m=50
                    level="HARD"
                    color=RED
                    start=False
        

    if r==None:
        r=10
    if c==None:
        c=10
    if m==None:
        m=10

    g=minesweeper(r,c,m,level,color)
    
    
    
    while(run):
        redrawWindow(WIN,g)
        for event in pygame.event.get():
            if event.type==pygame.QUIT:
                run=False
                pygame.quit()           
            if event.type==pygame.MOUSEBUTTONDOWN:
                pos=pygame.mouse.get_pos()

                if len(g.buttons)>1:
                    #yes
                    if g.buttons[1].isOverSquare(pos):
                        g.restart()
                        redrawWindow(WIN,g)
                    #no
                    elif g.buttons[2].isOverSquare(pos):
                        run=False
                        pygame.quit()  

                else:
                    if 100<=pos[1]<=600:
                        r,c=getGridPosition(pos,g)
                        if r>=0 and c>=0 and r<g.row and c<g.col:
                            mousePressed=pygame.mouse.get_pressed()
                            if mousePressed[0]:
                                g.dig(r,c)
                            elif mousePressed[2]:
                                if g.seenBoard[r][c]==' ':
                                    g.markFlag(r,c)
                                elif g.seenBoard[r][c]=='f':
                                    g.unmarkFlag(r,c)
    
            if len(g.buttons)>1:
                if event.type==pygame.MOUSEMOTION:
                    if g.buttons[1].isOverSquare(pos):
                        g.buttons[1].color=(153,255,153)
                    else:
                        g.buttons[1].color=GREEN
                    
                    if g.buttons[2].isOverSquare(pos):
                        g.buttons[2].color=(255,155,155)
                    else:
                        g.buttons[2].color=RED
                    

            



            
    pygame.quit()

if __name__ =="__main__":
    main()





game over
